In [176]:
#import statements
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from pandas import DataFrame
import matplotlib.pyplot # visualization library
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
stop_words = set(stopwords.words('english'))

import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import sys
import warnings
import matplotlib.pyplot as plt


from tensorflow import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers.convolutional import Conv1D    
from keras.optimizers import Adam

import seaborn as sns

In [161]:
reviews = pd.read_csv('Restaurant_Reviews.csv')
reviews

,Review,Liked,category
0,Wow... Loved this place.,1,0
1,Crust is not good.,0,2
2,Not tasty and the texture was just nasty.,0,2
3,Stopped by during the late May bank holiday of...,1,0
4,The selection on the menu was great and so wer...,1,"2, 3"
...,...,...,...
990,I think food should have flavor and texture an...,0,2
991,Appetite instantly gone.,0,2
992,Overall I was not impressed and would not go b...,0,0
993,"The whole experience was underwhelming, and I ...",0,0


In [162]:
#create a list of lists for category column
category_list = list(reviews.iloc[:,2])
#l = DataFrame(l)
#l = l.replace(' ', '')
#l = l.replace(',', '')
category_list = [list(x) for x in l]
for sub in category_list: 
    sub[:] = [ele for ele in sub if ele != ',']
for sub in category_list: 
    sub[:] = [ele for ele in sub if ele != ' ']
category_list = [[int(float(j)) for j in i] for i in category_list]
category_list

[[0],
 [2],
 [2],
 [0],
 [2, 3],
 [1],
 [2],
 [2],
 [2],
 [0],
 [1],
 [0],
 [0, 3],
 [2],
 [2, 4],
 [1],
 [0],
 [1],
 [0, 5],
 [0],
 [2],
 [2],
 [1],
 [1],
 [0],
 [2],
 [1, 2],
 [1, 2],
 [2],
 [3],
 [0],
 [0],
 [0],
 [0],
 [3],
 [2, 3],
 [1],
 [0],
 [1],
 [2],
 [0],
 [2],
 [1],
 [1, 2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [1],
 [2],
 [1],
 [2],
 [1, 2],
 [0, 5],
 [2],
 [2],
 [2],
 [1],
 [2],
 [2],
 [0],
 [0],
 [1, 2, 3],
 [0],
 [0],
 [1, 2],
 [0],
 [1, 2],
 [1],
 [1, 2],
 [0, 5],
 [2],
 [1],
 [2],
 [2],
 [2, 5],
 [0],
 [1],
 [2],
 [0],
 [2],
 [2],
 [2],
 [2],
 [2],
 [2],
 [0],
 [0],
 [0],
 [2],
 [1],
 [0],
 [0],
 [1],
 [0],
 [2],
 [1],
 [1],
 [2, 3],
 [2],
 [0],
 [0],
 [1, 2, 3],
 [1],
 [0],
 [0],
 [2],
 [1],
 [0],
 [2],
 [3],
 [0],
 [4],
 [1],
 [2],
 [0, 1],
 [2],
 [0, 1],
 [1],
 [2],
 [2],
 [1],
 [0, 2],
 [1, 2],
 [0],
 [2, 5],
 [2, 3],
 [2],
 [2],
 [1],
 [2],
 [0],
 [2],
 [2, 5],
 [1, 5],
 [1],
 [0],
 [1],
 [1, 2],
 [2],
 [2],
 [1, 4],
 [1, 2],
 [1],
 [0],
 [1],
 [1],
 [0, 5],
 [1]

In [163]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=lambda x: x, lowercase=False)
sparse_matrix = cv.fit_transform(category_list)

# To transform to dense matrix
sparse_matrix = sparse_matrix.todense()

sparse_matrix_df = pd.DataFrame(sparse_matrix)
sparse_matrix_df

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,0,0,1,0,0,0
2,0,0,1,0,0,0
3,1,0,0,0,0,0
4,0,0,1,1,0,0
...,...,...,...,...,...,...
990,0,0,1,0,0,0
991,0,0,1,0,0,0
992,1,0,0,0,0,0
993,1,0,0,0,0,0


In [164]:
reviews_wo_singlecol_category = reviews.iloc[:, 0:1]
reviews_wo_singlecol_category

,Review
0,Wow... Loved this place.
1,Crust is not good.
2,Not tasty and the texture was just nasty.
3,Stopped by during the late May bank holiday of...
4,The selection on the menu was great and so wer...
...,...
990,I think food should have flavor and texture an...
991,Appetite instantly gone.
992,Overall I was not impressed and would not go b...
993,"The whole experience was underwhelming, and I ..."


In [165]:
prepped_reviews = reviews_wo_singlecol_category.join(sparse_matrix_df)

In [166]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

X = []
sentences = list(reviews['Review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [168]:
categories = [0,1,2,3,4,5]
train, test = train_test_split(prepped_reviews, test_size=0.2, shuffle=True)
X_train = train.Review
X_test = test.Review

In [177]:
#LINEAR SVC
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing 0
Test accuracy is 0.8341708542713567
... Processing 1
Test accuracy is 0.8994974874371859
... Processing 2
Test accuracy is 0.8341708542713567
... Processing 3
Test accuracy is 0.9748743718592965
... Processing 4
Test accuracy is 0.964824120603015
... Processing 5
Test accuracy is 0.9798994974874372


In [178]:
#LOGISTIC REGRESSION
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing 0
Test accuracy is 0.8140703517587939
... Processing 1
Test accuracy is 0.8492462311557789
... Processing 2
Test accuracy is 0.7989949748743719
... Processing 3
Test accuracy is 0.9698492462311558
... Processing 4
Test accuracy is 0.9547738693467337
... Processing 5
Test accuracy is 0.9798994974874372


In [179]:
#NAIVE BAYES
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(X_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing 0
Test accuracy is 0.7788944723618091
... Processing 1
Test accuracy is 0.864321608040201
... Processing 2
Test accuracy is 0.7989949748743719
... Processing 3
Test accuracy is 0.9698492462311558
... Processing 4
Test accuracy is 0.9547738693467337
... Processing 5
Test accuracy is 0.9798994974874372
